In [9]:
%matplotlib inline

In [1]:
# Import python modules
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import netCDF4
import gc, sys, os
sys.path.append("../scripts")
import spatial_functions
import aem_utils
import netcdf_utils
import modelling_utils
import plotting_functions as plots
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
# Dash dependencies
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_table


Import the LCI


In [2]:
# The actual inversoin data are stored on disk as netcdf files. NetCDF is an efficient format for storing 
# self-describing containerised data. 
# The implementation of netcdf for AEM line data was done by Alex Ip using his geophys_utils package.
# https://github.com/GeoscienceAustralia/geophys_utils/tree/master/geophys_utils


root = r"C:\Users\symin\OneDrive\Documents\GA\AEM\LCI"

# Define path to the netcdf file
infile = os.path.join(root, "SouthernStuart_WB_MGA53.nc")

# Create an instance
lci = aem_utils.AEM_inversion(name = 'Laterally Contrained Inversion (LCI)',
                              inversion_type = 'deterministic',
                              netcdf_dataset = netCDF4.Dataset(infile))

# As these inversions have already been gridded we will add these raster datasets to the instance using the
# load_lci_layer_grid() function. This function belongs to the AEM_inversion class.

# Directory in which the grids are located
infile = os.path.join(root, "grids\\SSC_LCI_layer_grids.p")

# Run function
lci.load_lci_layer_grids_from_pickle(infile)

# Create polylines
lci.create_flightline_polylines()

Create an instance of the garjmcmctdem inversion and probe the results using the same syntax as above.

In [3]:
# Path to netcdf file
infile = r"C:\Users\symin\OneDrive\Documents\GA\AEM\rjmcmc\SSC_vanilla_rjmcmc_pmaps.nc"


# Create instance
rj = aem_utils.AEM_inversion(name = 'GARJMCMCTDEM',
                             inversion_type = 'stochastic',
                             netcdf_dataset = netCDF4.Dataset(infile))

In [4]:
# Now we have the lines we can grid the lci conductivity data onto vertical grids (known as sections)
# this is the easiest way to visualise the AEM conuctivity in 2-dimensions

# Assign the lci variables to grid
grid_vars = ['conductivity', 'data_residual', 'depth_of_investigation']


# Define the resolution of the sections
xres, yres = 40., 5.

# We will use the lines from the rj

lines = rj.data['line'][:]

# Define the output directory if saving the grids as hdf plots

hdf5_dir = r"C:\Temp\SSC_hdf5"

# if the directory doesn't exist, then create it
if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)  

# Gridding takes a few minutes so I pre-gridded them for you. The lci.grid_sections()
# function below will do the gridding for you. Instead we will use the load_sectoin_from_file()
# function, which loads hdf5 files produced using the grid_sections() function


#lci.grid_sections(variables = grid_vars, lines = lines, xres = xres, yres = yres,
#                  return_interpolated = True, save_hdf5 = True, hdf5_dir = hdf5_dir)

lci.load_sections_from_file(hdf5_dir, grid_vars, lines = lines)

In [5]:
# Grid the rj sections

# Assign the lci variables to grid
grid_vars = ['conductivity_p10', 'conductivity_p50', 'conductivity_p90', 'interface_depth_histogram']

# Define the resolution of the sections
xres, yres = 100., 2.

# We will use the lines from the rj

lines = np.unique(rj.data['line'][:].astype('int'))

# Define the output directory if saving the grids as hdf plots

hdf5_dir = r"C:\Temp\SSC_hdf5_rj"

# if the directory doesn't exist, then create it
if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)  

#rj.grid_sections(variables = grid_vars, lines = lines, xres = xres, yres = yres,
#                  return_interpolated = True, save_hdf5 = True, hdf5_dir = hdf5_dir)

rj.load_sections_from_file(hdf5_dir, grid_vars, lines = lines)

# 2. Visualise the data


# 2.1 Map plot

Our first standard plot is an plot of gridded conductivity with the points inverted with GARJMCMCTDEM
plotted as points.

# 3 Setup your model

Now we want to create a model boundary object for our interpreted surface. The easiest way to manage this is to have a separate instance the module boundary for every interface. This will allow us to produce eggs ready output files as we go.

In [6]:
# Create an modelled boundary instance

headings = ["inversion_name",'X', 'Y', 'ELEVATION', "DEM", "DEPTH", "UNCERTAINTY", "Type",
            "BoundaryNm", "BoundConf", "BasisOfInt", "OvrConf", "OvrStrtUnt", "OvrStrtCod", "UndStrtUnt",
           "UndStrtCod", "WithinType", "WithinStrt", "WithinStNo", "WithinConf", "InterpRef",
            "Comment", "SURVEY_LINE", "Operator"]

interp_file = r"C:\temp\Mereenie-Pertnjara_interface_interpreted_points.csv"

MP_surface = modelling_utils.modelled_boundary(name = 'Mereenie-Pertnjara interface',
                                               outfile_path = interp_file,
                                               interpreted_point_headings = headings)
MP_surface.interpreted_points = pd.read_csv(interp_file, index_col = 0)
# Define your surface
surface = MP_surface

# Assign attributes base on what you want to be
# entered into the eggs database
surface.Type = "BASE_Cenozoic_TOP_Paleozoic"
surface.OvrStrtUnt = "Pertnjara Formation"
surface.OvrStrtCod = 15098
surface.UndStrtUnt = "Mereenie Sandstone"
surface.UndStrtCod = 11667
surface.Inversion_name = "garjmcmtdem"
surface.BoundConf = "M"
surface.BasisOfInt = "IAEM"
surface.OvrConf = "M"
surface.InterpRef = ""
surface.Comment = ""
surface.Operator = "Neil Symington"
surface.WithinType = ""
surface.WithinStrt = ""
surface.WithinStNo = ""
surface.WithinConf = ""

In [20]:
def interp2scatter(line, gridded_data, interpreted_points, easting_col = 'X',
                   northing_col = 'Y', elevation_col = 'ELEVATION',
                   line_col = 'SURVEY_LINE'):

    mask = interpreted_points[line_col] == line
    utm_coords = np.column_stack((gridded_data[line]['easting'],
                                  gridded_data[line]['northing']))

    dist, inds = spatial_functions.nearest_neighbours(interpreted_points[mask][[easting_col,northing_col]].values,
                                                      utm_coords, max_distance=100.)

    grid_dists = gridded_data[line]['grid_distances'][inds]
    elevs = interpreted_points[mask][elevation_col].values
    fids = interpreted_points[mask].index
    return  grid_dists, elevs, fids

def dash_section(line, section_data, df_interp, colours):
    # Create the grid
    fig = px.imshow(img = np.log10(lci.section_data[line]['conductivity']),
                zmin = np.log10(0.001), zmax = np.log10(1.),
                x = lci.section_data[line]['grid_distances'],
                y = lci.section_data[line]['grid_elevations'],
        color_continuous_scale='viridis', aspect = 'auto'
    )

    interpx, interpz, fids = interp2scatter(line, section_data,
                                                              df_interp)

    labels = ["fiducial = " + str(x) for x in fids]

    fig.add_scatter(x = interpx,
                    y = interpz,
                    mode = 'markers',
                   hovertext = labels,
                    marker = {"color": colours})

    # Reverse y-axis
    fig.update_yaxes(autorange=True)

    return fig



In [32]:
# you may need extra dependencies to run these cells
import dash_bootstrap_components as dbc

line = lines[0]

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = surface.interpreted_points

app.layout = dbc.Container([
    dbc.Row(
                [
                    dbc.Col(html.H1("AEM interpretation dash board")),
                    dbc.Col(dbc.DropdownMenu(id = "section_dropdown",
                                            label="Section",
                                            children=[
                                                dbc.DropdownMenuItem("lci"),
                                                dbc.DropdownMenuItem("rjmcmc")
                                            ])),
                    dbc.Col(dbc.DropdownMenu(id = "line_dropdown",
                                            label="Line number",
                                            children=[
                                                dbc.DropdownMenuItem(100101),
                                                dbc.DropdownMenuItem(100201),
                                            ]))
                ]
                 ),
    dbc.Row(dbc.Col(id='section')),
    dbc.Row(dbc.Col(dash_table.DataTable(id='interp_table',
                                    columns=[{"name": i, "id": i} for i in df.columns],
                                    data=df.to_dict('records'),
                                 fixed_columns={ 'headers': True, 'data': 1 },
                                 sort_action="native",
                                 sort_mode="multi",
                                 column_selectable="multi",
                                 row_selectable="multi",
                                 row_deletable=True,
                                 selected_columns=[],
                                 selected_rows=[],
                                 style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                               'height': '70px'},
                                 style_cell={
                                     'backgroundColor': 'rgb(50, 50, 50)',
                                     'color': 'white',
                                     'minHeight': '50px',
                                     'minWidth': '0px', 'maxWidth': '800px',
                                     'whiteSpace': 'normal'
                                 },
                                  style_table={
                                              'maxHeight': '1000px',
                                              'overflowY': 'scroll',
                                              'maxWidth':  '1000px',
                                              'overflowX': 'scroll'})
             )),
],fluid=True)


@app.callback(
    Output('section', "children"),
    [Input('interp_table', "derived_virtual_data"),
     Input('interp_table', "derived_virtual_selected_rows")])
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colours = ['red' if i in derived_virtual_selected_rows else 'blue'
              for i in range(len(dff))]

    fig = dash_section(line, lci.section_data, dff, colours)

    return [
        dcc.Graph(
            id='graph',
            figure=fig
            ),
    ]

app.run_server(mode='external')



Dash app running on http://127.0.0.1:8050/


In [ ]:
surface.interpreted_points